In [1]:
%%bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O Miniconda3.sh
bash Miniconda3.sh -b -p /opt/conda
rm Miniconda3.sh

PREFIX=/opt/conda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /opt/conda


--2025-09-17 08:45:41--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162129736 (155M) [application/octet-stream]
Saving to: ‘Miniconda3.sh’

     0K .......... .......... .......... .......... ..........  0% 24.5M 6s
    50K .......... .......... .......... .......... ..........  0% 25.1M 6s
   100K .......... .......... .......... .......... ..........  0% 32.1M 6s
   150K .......... .......... .......... .......... ..........  0% 77.8M 5s
   200K .......... .......... .......... .......... ..........  0%  277M 4s
   250K .......... .......... .......... .......... ..........  0%  319M 3s
   300K .......... .......... .......... .......... ..........  0%  383M 3s
   350K .......... .......... .......... .......... .

After accepting the terms, you should be able to run the previous cells to create your environment and install packages.

In [2]:
%%bash
export PATH="/opt/conda/bin:$PATH"
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r


In [3]:
import sys
sys.path.append('/opt/conda/lib/python3.8/site-packages') # Adjust python version if needed

Après avoir exécuté ces cellules, vous devriez pouvoir utiliser la commande `conda` dans les cellules bash en ajoutant `export PATH="/opt/conda/bin:$PATH"` au début de vos commandes, ou en utilisant `!conda` directement dans les cellules de code Python.

Par exemple, pour créer un environnement :

In [4]:
%%bash
export PATH="/opt/conda/bin:$PATH"
conda create -n treeins_env_local python=3.8

conda activate treeins_env_local

conda install pytorch=1.9.0 torchvision==0.10.0 torchaudio==0.9.0 cudatoolkit=11.1 -c pytorch -c nvidia

pip install numpy==1.19.5

conda install openblas-devel -c anaconda

export CUDA_HOME=/usr/local/cuda-11

pip install -U git+https://github.com/NVIDIA/MinkowskiEngine -v --no-deps --install-option="--blas_include_dirs=${CONDA_PREFIX}/include" –install-option="--blas=openblas"

#CHECK IF TORCH AND MINKOWSKI ENGINE WORK AS EXPECTED:
(treeins_env_local) : python
Python 3.8.13 (default, #DATE#, #TIME#)
[GCC 7.5.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> torch.cuda.is_available()
True
>>> import MinkowskiEngine
>>> exit()
#CHECK FINISHED

pip install torch-scatter==2.0.8 -f https://data.pyg.org/whl/torch-1.9.0+cu111.html

pip install torch-sparse==0.6.12 -f https://data.pyg.org/whl/torch-1.9.0+cu111.html

pip install torch-geometric==1.7.2

#We got the file requirements.txt from here: https://github.com/nicolas-chaulet/torch-points3d/blob/master/requirements.txt but deleted the lines containing the following libraries in the file: torch, torchvision, torch-geometric,  torch-scatter, torch-sparse, numpy
pip install -r requirements.txt

pip install numba==0.55.1

conda install -c conda-forge hdbscan==0.8.27

conda install numpy-base==1.19.2

pip install joblib==1.1.0

Jupyter detected...
2 channel Terms of Service accepted
Retrieving notices: - \ | / - \ | / done
Channels:
 - defaults
Platform: linux-64
Solving environment: / done

## Package Plan ##

  environment location: /opt/conda/envs/treeins_env_local

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.9.9   |       h06a4308_0         127 KB
    libzlib-1.3.1              |       hb25bd0a_0          59 KB
    pip-24.2                   |   py38h06a4308_0         2.2 MB
    python-3.8.20              |       he870216_0        23.8 MB
    setuptools-75.1.0          |   py38h06a4308_0         1.7 MB
    wheel-0.44.0               |   py38h06a4308_0         108 KB
    zlib-1.3.1                 |       hb25bd0a_0          96 KB
    ------------------------------------------------------------
                            